In [6]:
!pip install kaggle
import kaggle



  Using cached kaggle-1.7.4.2-py3-none-any.whl.metadata (16 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached python_slugify-8.0.4-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached text_unidecode-1.3-py2.py3-none-any.whl.metadata (2.4 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
Using cached kaggle-1.7.4.2-py3-none-any.whl (173 kB)
Using cached urllib3-2.2.3-py3-none-any.whl (126 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached python_slugify-8.0.4-py2.py3-none-any.whl (10 kB)
Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)


In [7]:
from kaggle.api.kaggle_api_extended import KaggleApi

In [8]:
api = KaggleApi()
api.authenticate()

In [12]:
api.dataset_download_cli('jangedoo/utkface-new')

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors


In [13]:
import zipfile

In [14]:
with zipfile.ZipFile('utkface-new.zip', 'r') as zipref:
    zipref.extractall()

In [7]:
!pip install opencv-python


In [9]:
import os
import cv2
import numpy as np

# Define the path to the cropped images
data_dir = r"C:\VS CODE\ML\utkface_aligned_cropped\crop_part1"

# Function to load and process images with ethnicity labels
def load_images_from_folder(data_dir):
    images = []
    ethnicity_labels = []
    
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg"):
            # Construct the full file path
            file_path = os.path.join(data_dir, filename)
            
            try:
                # Load the image using OpenCV
                image = cv2.imread(file_path)
                if image is None:
                    print(f"Failed to load image: {file_path}")
                    continue
                
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
                # Extract the ethnicity (race) from the filename
                # Format is typically [age]_[gender]_[race]_[date&time].jpg
                parts = filename.split("_")
                if len(parts) >= 3:
                    ethnicity = int(parts[2])  # Ethnicity is the third element
                    
                    # Append the image and ethnicity label to the lists
                    images.append(image)
                    ethnicity_labels.append(ethnicity)
                else:
                    print(f"Filename format incorrect for {filename}")
            
            except Exception as e:
                print(f"Error processing {file_path}: {str(e)}")
    
    # Convert lists to numpy arrays for ML models
    images = np.array(images)
    ethnicity_labels = np.array(ethnicity_labels)
    
    return images, ethnicity_labels

# Load images and ethnicity labels
images, ethnicity_labels = load_images_from_folder(data_dir)
print(f"Loaded {len(images)} images with corresponding ethnicity labels.")

# Optionally, create a mapping for better understanding
ethnicity_mapping = {
    0: "White",
    1: "Black",
    2: "Asian",
    3: "Indian",
    4: "Others"
}

# Print distribution of ethnicities
for ethnicity_id, ethnicity_name in ethnicity_mapping.items():
    count = np.sum(ethnicity_labels == ethnicity_id)
    print(f"{ethnicity_name}: {count} images")

Error processing C:\VS CODE\ML\utkface_aligned_cropped\crop_part1\61_1_20170109142408075.jpg.chip.jpg: invalid literal for int() with base 10: '20170109142408075.jpg.chip.jpg'
Error processing C:\VS CODE\ML\utkface_aligned_cropped\crop_part1\61_3_20170109150557335.jpg.chip.jpg: invalid literal for int() with base 10: '20170109150557335.jpg.chip.jpg'
Loaded 9778 images with corresponding ethnicity labels.
White: 5265 images
Black: 405 images
Asian: 1553 images
Indian: 1452 images
Others: 1103 images


In [12]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Define the path to the cropped images
data_dir = r"C:\VS CODE\ML\utkface_aligned_cropped\crop_part1"

# Function to load & process a smaller dataset
def load_small_dataset(data_dir, max_images=5000, img_size=(64, 64)):  
    images, labels = [], []
    count = 0
    
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg") and count < max_images:
            file_path = os.path.join(data_dir, filename)
            img = cv2.imread(file_path)
            
            if img is None:
                continue
            
            img = cv2.resize(img, img_size)  # Resize to 64x64
            img = img / 255.0  # Normalize
            
            # Extract ethnicity from filename (format: age_gender_ethnicity_time.jpg)
            parts = filename.split("_")
            if len(parts) >= 3:
                try:
                    ethnicity = int(parts[2])
                    images.append(img)
                    labels.append(ethnicity)
                    count += 1
                except ValueError:
                    continue  # Skip invalid labels
    
    return np.array(images), np.array(labels)

# Load reduced dataset
images, labels = load_small_dataset(data_dir)

# Split data into train & test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, stratify=labels, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train_categorical = to_categorical(y_train, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)

print(f"Loaded {len(X_train)} training samples and {len(X_test)} testing samples.")


Loaded 4000 training samples and 1000 testing samples.


In [32]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from collections import Counter

# Define the path to the cropped images
data_dir = r"C:\VS CODE\ML\utkface_aligned_cropped\crop_part1"

# Function to load & process dataset with under-sampling
def load_and_under_sample_dataset(data_dir, max_images_per_class=500):  
    images, labels = [], []
    
    # Dictionary to store image lists per ethnicity
    ethnicity_dict = {0: [], 1: [], 2: [], 3: [], 4: []}  # 0: White, 1: Black, 2: Asian, 3: Indian, 4: Others

    # Load images and categorize them by ethnicity
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg"):
            file_path = os.path.join(data_dir, filename)
            img = cv2.imread(file_path)
            
            if img is None:
                continue
            
            img = cv2.resize(img, (64, 64))  # Resize to 64x64
            img = img / 255.0  # Normalize
            
            # Extract ethnicity from filename (format: age_gender_ethnicity_time.jpg)
            parts = filename.split("_")
            if len(parts) >= 3:
                try:
                    ethnicity = int(parts[2])
                    ethnicity_dict[ethnicity].append(img)  # Add image to the corresponding ethnicity list
                except ValueError:
                    continue  # Skip invalid labels
    
    # Now, under-sample each class to have a max of `max_images_per_class`
    for ethnicity in ethnicity_dict:
        if len(ethnicity_dict[ethnicity]) > max_images_per_class:
            ethnicity_dict[ethnicity] = ethnicity_dict[ethnicity][:max_images_per_class]  # Limit the samples
    
    # Flatten lists of images and labels
    for ethnicity in ethnicity_dict:
        images.extend(ethnicity_dict[ethnicity])
        labels.extend([ethnicity] * len(ethnicity_dict[ethnicity]))  # Repeat ethnicity label for each image
    
    return np.array(images), np.array(labels)

# Load dataset with under-sampling
images, labels = load_and_under_sample_dataset(data_dir)

# Split data into train & test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, stratify=labels, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train_categorical = to_categorical(y_train, num_classes)
y_test_categorical = to_categorical(y_test, num_classes)

print(f"Loaded {len(X_train)} training samples and {len(X_test)} testing samples.")
print(f"Class distribution in training set: {dict(Counter(y_train))}")


Loaded 1924 training samples and 481 testing samples.
Class distribution in training set: {np.int64(0): 400, np.int64(4): 400, np.int64(2): 400, np.int64(3): 400, np.int64(1): 324}


In [33]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Simple CNN Model
def build_light_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')  # Multi-class classification
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Get input shape (64x64x3 for RGB)
input_shape = X_train.shape[1:]
model = build_light_cnn(input_shape, num_classes)

# Model summary
model.summary()


c:\VS CODE\ML\FeatureDetection\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 62, 62, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 31, 31, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 29, 29, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       401,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 406,885 (1.55 MB)

 Trainable params: 406,885 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize ImageDataGenerator with augmentation for minority classes
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Fit the data generator on your training data (X_train)
datagen.fit(X_train)

# Now, train the model using the augmented data
history = model.fit(
    datagen.flow(X_train, y_train_categorical, batch_size=32),
    epochs=10,
    validation_data=(X_test, y_test_categorical)
)


Epoch 1/10


c:\VS CODE\ML\FeatureDetection\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


61/61 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.2185 - loss: 1.6084 - val_accuracy: 0.3721 - val_loss: 1.4982
Epoch 2/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.3201 - loss: 1.5194 - val_accuracy: 0.4220 - val_loss: 1.4302
Epoch 3/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.3480 - loss: 1.4768 - val_accuracy: 0.3514 - val_loss: 1.4782
Epoch 4/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.3331 - loss: 1.4823 - val_accuracy: 0.4428 - val_loss: 1.3674
Epoch 5/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.3712 - loss: 1.4756 - val_accuracy: 0.4200 - val_loss: 1.3766
Epoch 6/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.3712 - loss: 1.4424 - val_accuracy: 0.4491 - val_loss: 1.3639
Epoch 7/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.3940 - loss: 1.4122 - val_accuracy: 0.4241 - val_loss: 1.3713
Epoch 8/10
61/61 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.3866 - loss: 1.4245 - val_accuracy: 0.4740 - val_loss: 1.

In [35]:
test_loss, test_acc = model.evaluate(X_test, y_test_categorical)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5234 - loss: 1.2703
Test Accuracy: 49.48%


In [36]:
model.save('C:\VS CODE\ML\Models\ethnicity_model.h5')

In [39]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('C:\VS CODE\ML\Models\ethnicity_model.h5')
print("Model loaded successfully.")




# Function to load and preprocess the image
def predict_ethnicity(image_path):
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    
    # Check if the image is successfully loaded
    if img is None:
        print(f"Error: Could not read the image from path: {image_path}")
        return None
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = cv2.resize(img, (64,64))  # Resize to match the input size for the model
    img = img / 255.0  # Normalize the pixel values to [0, 1]
    
    img = np.expand_dims(img, axis=0)  # Add batch dimension (shape becomes (1, 128, 128, 3))
    
    # Predict the ethnicity
    predictions = model.predict(img)
    predicted_class = np.argmax(predictions, axis=1)[0]  # Get the class with the highest probability
    
    return ethnicity_mapping[predicted_class]

# Test the function with a new image
image_path = 'asianperson2.jpeg'  # Replace with the path to your image
predicted_ethnicity = predict_ethnicity(image_path)
print(f"The predicted ethnicity is: {predicted_ethnicity}")


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
The predicted ethnicity is: Black
